# Training a Chatbot from own data

In [1]:
#Frameworks & librearies

import random #Choose random response
import json   #read train data
import pickle
import numpy as np
import nltk
import spacy #working -> work
from nltk.stem import SnowballStemmer
from tensorflow.keras.models import Sequential #Deep Learning
from tensorflow.keras.layers import Dense,Activation, Dropout  #Layers NN
from tensorflow.keras.optimizers import SGD #Gradient Descent

In [2]:
#nltk.download('punkt')

## Lematización

En español, por ejemplo, sabemos que canto, cantas, canta, cantamos, cantáis, cantan son distintas formas (conjugaciones) de un mismo verbo (cantar)

In [3]:
import spacy


nlp = spacy.load("es_core_news_sm") #Spanish

#nlp=spacy.load("en_core_web_sm") #English


def normalize(text,
              tildes={'á':'a','é':'e','í':'i','ó':'o','ú':'u'},
             save_words=['dias','donde']):
    
    #Remove tildes
    text=text.lower()
    for k,v in tildes.items():
        text=text.replace(k,v)
    #spacy model
    doc = nlp(text)
    #drop stopwords and punctuation
    words=[]
    for t in doc:
        if not t.is_punct | t.is_stop:
            words.append(t.orth_)
        elif not t.is_punct and t.orth_ in save_words:
            words.append(t.orth_)
            
    #words = [t.orth_ for t in doc if not t.is_punct | t.is_stop] 
    lexical_tokens = [t.lower() for t in words if len(t) > 2 and t.isalpha()]
    return " ".join(lexical_tokens)


#Lemmatizer
def lemmatizer(text):
    doc = nlp(text)
    lemmas = [tok.lemma_.lower() for tok in doc]
    return lemmas   
        
normText = normalize("Soy un texto de prueba.Buenos dias. hola, como estas? ¿Cuántos tokens me quedarán después de la normalización?")
lemmatizer(normText)

['texto', 'dia', 'holar', 'tokens', 'quedar', 'normalizacion']

In [4]:
#Load Data
intents=json.loads(open("intents.json").read())
#intents

In [5]:
#Get Documents to training

words=[]
classes=[]
documents=[]
ignore_letters=['?','!',"¡","¿",",","."]

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
#documents
#words

In [6]:
words

['hola',
 'hola',
 ',',
 'como',
 'estas',
 '?',
 'buenos',
 'dias',
 'buenas',
 'tardes',
 'buenas',
 'noches',
 'holi',
 'muy',
 'buenas',
 'chao',
 'adios',
 'hasta',
 'pronto',
 'muchas',
 'gracias',
 'gracias',
 'te',
 'agradezco',
 'vemos',
 'donde',
 'donde',
 'puedo',
 'comprar',
 'donde',
 'los',
 'puedo',
 'encontrar',
 'donde',
 'puedo',
 'encontrar',
 'la',
 'tienda',
 'donde',
 'estan',
 'ubicados',
 'donde',
 'los',
 'puedo',
 'ubicar',
 'lugares',
 'fisicos',
 'a',
 'donde',
 'puedo',
 'ir',
 'como',
 'es',
 'la',
 'direccion',
 'en',
 'la',
 'ciudad',
 'donde',
 'puedo',
 'encontrarlos',
 'donde',
 'puedo',
 'encontrarlos',
 'en',
 'cual',
 'calle',
 'se',
 'encuentran',
 'como',
 'es',
 'el',
 'menu',
 'venden',
 'que',
 'producto',
 'tienen',
 'a',
 'la',
 'venta',
 'que',
 'puedo',
 'comprar',
 'que',
 'hay',
 'en',
 'el',
 'menu',
 'catalogo',
 'quiero',
 'comprar',
 'algo',
 'me',
 'gustaria',
 'tener',
 'mas',
 'informacion',
 'quiero',
 'comprar',
 'productos',
 

In [7]:
#lematize word list
normWords=normalize(" ".join(words))
words=[lemma for lemma in lemmatizer(normWords) if lemma not in ignore_letters]

In [8]:
#Remove duplicates
words=sorted(set(words))
words

['abierto',
 'abrir',
 'adio',
 'agradecer',
 'atender',
 'calle',
 'catalogo',
 'cerrar',
 'chao',
 'ciudad',
 'comprar',
 'dia',
 'direccion',
 'disponible',
 'domingo',
 'donde',
 'encontrar',
 'encontrar él',
 'encontrarlos',
 'fin',
 'fisico',
 'galleta',
 'gracias',
 'gustaria',
 'hola',
 'holar',
 'holi',
 'hora',
 'horario',
 'informacion',
 'lugar',
 'lunes',
 'menu',
 'noche',
 'producto',
 'querer',
 'sabado',
 'semana',
 'tarde',
 'tienda',
 'ubicado',
 'ubicar',
 'vender',
 'venta',
 'ventar',
 'ver',
 'viernes']

In [9]:
#remove duplicates classes
classes=sorted(set(classes))

In [10]:
#Save lists in picke format
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [11]:
#Create Matrix for train
training=[]
output_empty=[0 for c in classes]



for document in documents:
    bag=[] #[0,0,0,1] if words in doc
    word_patterns=document[0]
    word_patterns=[lemma for lemma in lemmatizer(" " .join(word_patterns)) if lemma not in ignore_letters]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1 #index class-> id for labels
    training.append([bag,output_row])  #Xi=[xi,yi] |xi=[0,0,1,0], yi=[2]

#training[:2]

In [12]:
#Random X
random.shuffle(training)

In [13]:
training=np.array(training)
train_x=list(training[:,0])  
train_y=list(training[:,1])

c:\users\usuario\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [14]:
#dimmensions
n_features=len(train_x[0])
n_outputs=len(train_y[0])
n_features,n_outputs

(47, 5)

In [15]:
#Model from Keras

model=Sequential()
model.add(Dense(64,
                input_shape=(n_features,),
                activation='relu'           
               )) #full connected ->64 neurons, relu function max(0,n)
model.add(Dropout(0.5)) #Evit overfitting shutoff neurons with probably .5
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(
    n_outputs,
    activation='softmax'
)) #Probably [.3,.8,.7] for each class


#Stocastic Gradient Descent [Optimizer]
sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)

#Compile Model
model.compile(loss='categorical_crossentropy',
             optimizer=sgd,metrics=['accuracy'])

In [16]:
#Traing (Fit) Model

X=np.array(train_x)
y=np.array(train_y)

epochs=100 #Number times to show data to model
batch_size=5 #5 rows at time for epoch
verbose=1 #Minimun information

hist=model.fit(X,y,epochs=epochs,batch_size=batch_size,verbose=verbose)

Epoch 1/100
10/10 [==============================] - 1s 2ms/step - loss: 1.6343 - accuracy: 0.2497
Epoch 2/100
10/10 [==============================] - 0s 889us/step - loss: 1.5894 - accuracy: 0.2168
Epoch 3/100
10/10 [==============================] - 0s 778us/step - loss: 1.5335 - accuracy: 0.3459
Epoch 4/100
10/10 [==============================] - 0s 778us/step - loss: 1.4733 - accuracy: 0.4893
Epoch 5/100
10/10 [==============================] - 0s 889us/step - loss: 1.3943 - accuracy: 0.5346
Epoch 6/100
10/10 [==============================] - 0s 1ms/step - loss: 1.3868 - accuracy: 0.4024
Epoch 7/100
10/10 [==============================] - 0s 778us/step - loss: 1.3198 - accuracy: 0.5596
Epoch 8/100
10/10 [==============================] - 0s 1000us/step - loss: 1.3136 - accuracy: 0.5572
Epoch 9/100
10/10 [==============================] - 0s 889us/step - loss: 1.2299 - accuracy: 0.5142
Epoch 10/100
10/10 [==============================] - 0s 889us/step - loss: 1.2090 - accuracy:

10/10 [==============================] - 0s 889us/step - loss: 0.1746 - accuracy: 0.9927
Epoch 82/100
10/10 [==============================] - 0s 889us/step - loss: 0.1204 - accuracy: 0.9677
Epoch 83/100
10/10 [==============================] - 0s 778us/step - loss: 0.1665 - accuracy: 0.9526
Epoch 84/100
10/10 [==============================] - 0s 778us/step - loss: 0.1736 - accuracy: 0.9828
Epoch 85/100
10/10 [==============================] - 0s 889us/step - loss: 0.3235 - accuracy: 0.8908
Epoch 86/100
10/10 [==============================] - 0s 778us/step - loss: 0.1909 - accuracy: 0.9621
Epoch 87/100
10/10 [==============================] - 0s 667us/step - loss: 0.2219 - accuracy: 0.9252
Epoch 88/100
10/10 [==============================] - 0s 1ms/step - loss: 0.2379 - accuracy: 0.9062
Epoch 89/100
10/10 [==============================] - 0s 889us/step - loss: 0.2034 - accuracy: 0.9631
Epoch 90/100
10/10 [==============================] - 0s 889us/step - loss: 0.1629 - accuracy: 0.

In [17]:
#Save model
model.save('chatbotmodel.h5',hist)

# Make Chatbot

In [31]:
import json
import pickle
import numpy as np
import nltk
from tensorflow.keras.models import load_model
import spacy

#load Spacy Model
#nlp = spacy.load("en_core_web_sm") #English
nlp = spacy.load("es_core_news_sm") #Spanish

#load pickles
words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))

#load data (intents) json
intents=json.loads(open("intents.json",encoding="utf-8").read())

#Load Model
model=load_model('chatbotmodel.h5')
len(classes),len(words)

(5, 47)

In [32]:
#Clean Text

def clean_sentence(sentence,ignore_letters=['?','!',"¡","¿",",","."]):
    #Tokenize
    sentence_words=sentence.split()
    #Lematize and normalize
    normWords=normalize(" " .join(sentence_words))
    sentence_words=[lemma for lemma in lemmatizer(normWords) if lemma not in ignore_letters]
    if sentence_words==[]:
        return sentence.lower().split()
    return sentence_words

clean_sentence("hola, como estás?")

['holar']

In [33]:
#Matrix from text 

"""
Matriz del mismo tamaño que la usada para entrenar x[i,j].

si x[i,j] = 1 la palabra j de la oración i SI está en la lista de palabras de 
    entrenamiento (features)
    
si x[i,j] = 0 la palabra j de la oración i NO está en la lista de palabras de 
    entrenamiento (features)
"""

def bag_of_words(sentence):
    sentence_words=clean_sentence(sentence)
    bag=[0]* len(words)
    for w in sentence_words:
        for i,word in enumerate(words):
            if word==w:
                bag[i]=1
    return np.array(bag)

bag_of_words("hola como estas, eres un bot?")              

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [34]:
#Predict Class from sentence

def predict_class(sentence):
    
    
    bow=bag_of_words(sentence)
    x=np.array([bow])
    res=model.predict(x)[0]
    results=[[i,r] for i,r in enumerate(res)]
    results.sort(key=lambda x: x[1],reverse=True)
    return_list=[]
    for r in results:
        _data={'intent':classes[r[0]],'probability':r[1]}
        return_list.append(_data)
        return return_list


result=predict_class("hola como estas, eres un bot?")
result

[{'intent': 'greeting', 'probability': 0.9212816}]

In [35]:
#Make a Response

def get_response(intents_list,intents_json,ERROR_THRESHOLD=0.7):
    """
    intentens_list: list prediction od model,
    intents_json: own data (json file)
    ERROR_THRESHOL: Minimum probablity to give a response
    """
    print(intents_list[0]['probability'])
    if intents_list[0]['probability']<ERROR_THRESHOLD:
        return "Lo siento, no entiendo lo que me dices, pero puedes preguntarme por el menu!"
    
    
    tag=intents_list[0]['intent'] #get intent mayor probability
    list_intents=intents_json['intents']
    for i in list_intents:
        if i['tag']==tag:
            results=random.choice(i['responses']) #Random response from responses
            break
    return results



In [36]:
while True:
    message=input("yo: ")
    ints=predict_class(message)
    res=get_response(ints,intents)
    print("bot:",res)
    if message=="quit":
        break

yo: hola
0.9212816
bot: Hola!
yo: buenos días
0.8963507
bot: Hola, un gusto conocerte
yo: hola, esta abierto?
0.9895839
bot: Abrimos de Lunes a Domingo. 6am-10pm
yo: mmm y que productos venden?
0.9999999
bot: Tenemos Galletas Chocolate a $2000, Galletas de Coco a $9999 y nuestros cafés todos a $50
yo: en que horario?
0.99952435
bot: Abrimos de Lunes a Domingo. 6am-10pm
yo: me puedes dar una direccion?
0.9990934
bot: Puedes encontrarnos en la Calle 100, al lado del Starbucks. Te esperamos.
yo: dame la ubicacion
0.7208332
bot: Adios :(
yo: donde estan ubicados
1.0
bot: Puedes encontrarnos en la Calle 100, al lado del Starbucks. Te esperamos.
yo: adios
0.90221465
bot: Chao, un gusto conocerte!
yo: quit
0.7208332
bot: Vuelve pronto!
